# Voxtral HF Plugin

> Plugin implementation for Mistral Voxtral transcription through Hugging Face Transformers

In [ ]:
#| default_exp plugin

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import json
import logging
from pathlib import Path
from typing import Dict, Any, Optional, List, Union, Generator
import tempfile
import warnings
from threading import Thread

from fastcore.basics import patch

import numpy as np
import torch
import soundfile as sf

try:
    from transformers import VoxtralForConditionalGeneration, AutoProcessor
    from transformers import TextStreamer, TextIteratorStreamer
    VOXTRAL_AVAILABLE = True
except ImportError:
    VOXTRAL_AVAILABLE = False
    
from cjm_transcription_plugin_system.plugin_interface import PluginInterface
from cjm_transcription_plugin_system.core import AudioData, TranscriptionResult

In [ ]:
#| export
class VoxtralHFPlugin(PluginInterface):
    """Mistral Voxtral transcription plugin via Hugging Face Transformers."""
    
    def __init__(self):
        """Initialize the Voxtral HF plugin with default configuration."""
        self.logger = logging.getLogger(f"{__name__}.{type(self).__name__}")
        self.config = {}
        self.model = None
        self.processor = None
        self.device = None
        self.dtype = None
    
    @property
    def name(
        self
    ) -> str:  # Returns the plugin name
        """Get the plugin name identifier."""
        return "voxtral_hf"
    
    @property
    def version(
        self
    ) -> str:  # Returns the plugin version
        """Get the plugin version string."""
        return "1.0.0"
    
    @property
    def supported_formats(
        self
    ) -> List[str]:  # Returns list of supported audio formats
        """Get the list of supported audio file formats."""
        return ["wav", "mp3", "flac", "m4a", "ogg", "webm", "mp4", "avi", "mov"]
    
    def get_config_schema(
        self
    ) -> Dict[str, Any]:  # Returns the configuration schema dictionary
        """Return configuration schema for Voxtral HF."""
        return {
            "$schema": "http://json-schema.org/draft-07/schema#",
            "type": "object",
            "title": "Voxtral HF Configuration",
            "properties": {
                "model_id": {
                    "type": "string",
                    "enum": ["mistralai/Voxtral-Mini-3B-2507", "mistralai/Voxtral-Small-24B-2507"],
                    "default": "mistralai/Voxtral-Mini-3B-2507",
                    "description": "Voxtral model to use. Mini is faster, Small is more accurate."
                },
                "device": {
                    "type": "string",
                    "enum": ["auto", "cpu", "cuda"],
                    "default": "auto",
                    "description": "Device for inference (auto will use CUDA if available)"
                },
                "dtype": {
                    "type": "string",
                    "enum": ["auto", "bfloat16", "float16", "float32"],
                    "default": "auto",
                    "description": "Data type for model weights (auto will use bfloat16 on GPU, float32 on CPU)"
                },
                "language": {
                    "type": ["string", "null"],
                    "default": "en",
                    "description": "Language code for transcription (e.g., 'en', 'es', 'fr')",
                    "examples": ["en", "es", "fr", "de", "it", "pt", "nl", "pl", "ru", "zh", "ja", "ko"]
                },
                "max_new_tokens": {
                    "type": "integer",
                    "minimum": 1,
                    "maximum": 50000,
                    "default": 25000,
                    "description": "Maximum number of tokens to generate"
                },
                "do_sample": {
                    "type": "boolean",
                    "default": False,
                    "description": "Whether to use sampling (true) or greedy decoding (False)"
                },
                "temperature": {
                    "type": "number",
                    "minimum": 0.0,
                    "maximum": 2.0,
                    "default": 1.0,
                    "description": "Temperature for sampling (only used when do_sample=true)"
                },
                "top_p": {
                    "type": "number",
                    "minimum": 0.0,
                    "maximum": 1.0,
                    "default": 0.95,
                    "description": "Top-p (nucleus) sampling parameter (only used when do_sample=true)"
                },
                "streaming": {
                    "type": "boolean",
                    "default": False,
                    "description": "Enable streaming output (yields partial results as they're generated)"
                },
                "trust_remote_code": {
                    "type": "boolean",
                    "default": False,
                    "description": "Whether to trust remote code when loading the model"
                },
                "cache_dir": {
                    "type": ["string", "null"],
                    "default": None,
                    "description": "Directory to cache downloaded models"
                },
                "compile_model": {
                    "type": "boolean",
                    "default": False,
                    "description": "Use torch.compile for potential speedup (requires PyTorch 2.0+)"
                },
                "load_in_8bit": {
                    "type": "boolean",
                    "default": False,
                    "description": "Load model in 8-bit quantization (requires bitsandbytes)"
                },
                "load_in_4bit": {
                    "type": "boolean",
                    "default": False,
                    "description": "Load model in 4-bit quantization (requires bitsandbytes)"
                }
            },
            "required": ["model_id"],
            "additionalProperties": False
        }
    
    def get_current_config(
        self
    ) -> Dict[str, Any]:  # Returns the current configuration dictionary
        """Return current configuration."""
        defaults = self.get_config_defaults()
        return {**defaults, **self.config}
    
    def initialize(
        self,
        config: Optional[Dict[str, Any]] = None  # Configuration dictionary to initialize the plugin
    ) -> None:
        """Initialize the plugin with configuration."""
        if config:
            is_valid, error = self.validate_config(config)
            if not is_valid:
                raise ValueError(f"Invalid configuration: {error}")
        
        # Merge with defaults
        defaults = self.get_config_defaults()
        self.config = {**defaults, **(config or {})}
        
        # Set device
        if self.config["device"] == "auto":
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
        else:
            self.device = self.config["device"]
        
        # Set dtype
        if self.config["dtype"] == "auto":
            if self.device == "cuda":
                self.dtype = torch.bfloat16
            else:
                self.dtype = torch.float32
        else:
            dtype_map = {
                "bfloat16": torch.bfloat16,
                "float16": torch.float16,
                "float32": torch.float32
            }
            self.dtype = dtype_map[self.config["dtype"]]
        
        self.logger.info(f"Initialized Voxtral HF plugin with model '{self.config['model_id']}' on device '{self.device}' with dtype '{self.dtype}'")
    
    def _load_model(
        self
    ) -> None:
        """Load the Voxtral model and processor (lazy loading)."""
        if self.model is None or self.processor is None:
            try:
                self.logger.info(f"Loading Voxtral model: {self.config['model_id']}")
                
                # Load processor
                self.processor = AutoProcessor.from_pretrained(
                    self.config["model_id"],
                    cache_dir=self.config.get("cache_dir"),
                    trust_remote_code=self.config.get("trust_remote_code", False)
                )
                
                # Model loading kwargs
                model_kwargs = {
                    "cache_dir": self.config.get("cache_dir"),
                    "trust_remote_code": self.config.get("trust_remote_code", False),
                    "device_map": self.device,
                }
                
                # Add quantization settings if specified
                if self.config.get("load_in_8bit", False):
                    model_kwargs["load_in_8bit"] = True
                elif self.config.get("load_in_4bit", False):
                    model_kwargs["load_in_4bit"] = True
                else:
                    model_kwargs["dtype"] = self.dtype
                
                # Load model
                self.model = VoxtralForConditionalGeneration.from_pretrained(
                    self.config["model_id"],
                    **model_kwargs
                )
                
                # Optionally compile the model (PyTorch 2.0+)
                if self.config.get("compile_model", False) and hasattr(torch, 'compile'):
                    self.model = torch.compile(self.model)
                    self.logger.info("Model compiled with torch.compile")
                    
                self.logger.info("Voxtral model loaded successfully")
            except Exception as e:
                raise RuntimeError(f"Failed to load Voxtral model: {e}")
    
    def _prepare_audio(
        self,
        audio: Union[AudioData, str, Path]  # Audio data, file path, or Path object to prepare
    ) -> str:  # Returns path to the prepared audio file
        """Prepare audio for Voxtral processing."""
        if isinstance(audio, (str, Path)):
            # Already a file path
            return str(audio)
        
        elif isinstance(audio, AudioData):
            # Save AudioData to temporary file
            with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp_file:
                # Ensure audio is in the correct format
                audio_array = audio.samples
                
                # If stereo, convert to mono
                if audio_array.ndim > 1:
                    audio_array = audio_array.mean(axis=1)
                
                # Ensure float32 and normalized
                if audio_array.dtype != np.float32:
                    audio_array = audio_array.astype(np.float32)
                
                # Normalize if needed
                if audio_array.max() > 1.0:
                    audio_array = audio_array / np.abs(audio_array).max()
                
                # Save to file
                sf.write(tmp_file.name, audio_array, audio.sample_rate)
                return tmp_file.name
        else:
            raise ValueError(f"Unsupported audio input type: {type(audio)}")
    
    def execute(
        self,
        audio: Union[AudioData, str, Path],  # Audio data or path to audio file to transcribe
        **kwargs #  Additional arguments to override config
    ) -> TranscriptionResult:  # Returns transcription result with text and metadata
        """Transcribe audio using Voxtral."""
        # Load model if not already loaded
        self._load_model()
        
        # Prepare audio file
        audio_path = self._prepare_audio(audio)
        temp_file_created = not isinstance(audio, (str, Path))
        
        try:
            # Merge runtime kwargs with config
            exec_config = {**self.config, **kwargs}
            
            # Prepare inputs
            self.logger.info(f"Processing audio with Voxtral {exec_config['model_id']}")
            
            inputs = self.processor.apply_transcription_request(
                language=exec_config.get("language", "en"),
                audio=str(audio_path),
                model_id=exec_config["model_id"]
            )
            inputs = inputs.to(self.device, dtype=self.dtype)
            
            # Generation kwargs
            generation_kwargs = {
                "max_new_tokens": exec_config.get("max_new_tokens", 25000),
                "do_sample": exec_config.get("do_sample", False),
            }
            
            # Add sampling parameters if sampling is enabled
            if generation_kwargs["do_sample"]:
                generation_kwargs["temperature"] = exec_config.get("temperature", 1.0)
                generation_kwargs["top_p"] = exec_config.get("top_p", 0.95)
            
            # Generate transcription
            with torch.no_grad():
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    
                    outputs = self.model.generate(
                        **inputs,
                        **generation_kwargs
                    )
            
            # Decode the output
            result_text = self.processor.batch_decode(
                outputs[:, inputs.input_ids.shape[1]:], 
                skip_special_tokens=True
            )[0]

            # Clean up tensors immediately
            del inputs
            del outputs
            
            # Clear GPU cache if using CUDA
            if self.device == "cuda" and torch.cuda.is_available():
                torch.cuda.empty_cache()
            
            # Create transcription result
            transcription_result = TranscriptionResult(
                text=result_text.strip(),
                confidence=None,  # Voxtral doesn't provide confidence scores
                segments=None,  # Voxtral doesn't provide segments by default
                metadata={
                    "model": exec_config["model_id"],
                    "language": exec_config.get("language", "en"),
                    "device": self.device,
                    "dtype": str(self.dtype),
                }
            )
            
            self.logger.info(f"Transcription completed: {len(result_text.split())} words")
            return transcription_result
            
        finally:
            # Clean up temporary file if created
            if temp_file_created:
                try:
                    Path(audio_path).unlink()
                except Exception:
                    pass

    def is_available(
        self
    ) -> bool:  # Returns True if Voxtral and its dependencies are available
        """Check if Voxtral is available."""
        return VOXTRAL_AVAILABLE
    
    def cleanup(
        self
    ) -> None:
        """Clean up resources with aggressive memory management."""
        if self.model is None and self.processor is None:
            self.logger.info("No models to clean up")
            return
        
        self.logger.info("Unloading Voxtral model")
        
        try:
            # Move model to CPU first if it's on GPU (frees GPU memory immediately)
            if self.model is not None and self.device == "cuda":
                try:
                    # Move to CPU to free GPU memory
                    self.model = self.model.to('cpu')
                    self.logger.debug("Model moved to CPU")
                except Exception as e:
                    self.logger.warning(f"Could not move model to CPU: {e}")
            
            # Delete processor first (it may hold references to model components)
            if self.processor is not None:
                del self.processor
                self.processor = None
                self.logger.debug("Processor deleted")
            
            # Delete model
            if self.model is not None:
                del self.model
                self.model = None
                self.logger.debug("Model deleted")
            
            # Force garbage collection BEFORE GPU operations
            import gc
            gc.collect()
            
            # GPU-specific cleanup
            if self.device == "cuda" and torch.cuda.is_available():
                # Empty cache and synchronize
                torch.cuda.empty_cache()
                torch.cuda.synchronize()
                
                # Optional: more aggressive cleanup
                torch.cuda.ipc_collect()
                
                # Log memory stats
                if torch.cuda.is_available():
                    allocated = torch.cuda.memory_allocated() / 1024**3
                    reserved = torch.cuda.memory_reserved() / 1024**3
                    self.logger.info(f"GPU memory after cleanup - Allocated: {allocated:.2f}GB, Reserved: {reserved:.2f}GB")
            
            self.logger.info("Cleanup completed successfully")
            
        except Exception as e:
            self.logger.error(f"Error during cleanup: {e}")
            # Ensure references are cleared even if cleanup fails
            self.model = None
            self.processor = None

In [ ]:
#| export
@patch
def supports_streaming(
    self:VoxtralHFPlugin
) -> bool:
    """Check if this plugin supports streaming transcription."""
    return True

@patch
def execute_stream(
    self:VoxtralHFPlugin,
    audio: Union[AudioData, str, Path],  # Audio data or path to audio file
    **kwargs  # Additional plugin-specific parameters
) -> Generator[str, None, TranscriptionResult]:  # Yields text chunks, returns final result
    """Stream transcription results chunk by chunk.
    
    Args:
        audio: Audio data or path to audio file
        **kwargs: Additional plugin-specific parameters
        
    Yields:
        str: Partial transcription text chunks as they become available
        
    Returns:
        TranscriptionResult: Final complete transcription with metadata
    """
    # Load model if not already loaded
    self._load_model()
    
    # Prepare audio file
    audio_path = self._prepare_audio(audio)
    temp_file_created = not isinstance(audio, (str, Path))
    
    try:
        # Merge runtime kwargs with config
        exec_config = {**self.config, **kwargs}
        
        # Prepare inputs
        self.logger.info(f"Streaming transcription with Voxtral {exec_config['model_id']}")
        
        inputs = self.processor.apply_transcription_request(
            language=exec_config.get("language", "en"),
            audio=str(audio_path),
            model_id=exec_config["model_id"]
        )
        inputs = inputs.to(self.device, dtype=self.dtype)
        
        # Create streamer
        from transformers import TextIteratorStreamer
        streamer = TextIteratorStreamer(
            self.processor.tokenizer, 
            skip_prompt=True, 
            skip_special_tokens=True
        )
        
        # Generation kwargs
        generation_kwargs = {
            **inputs,
            "max_new_tokens": exec_config.get("max_new_tokens", 25000),
            "do_sample": exec_config.get("do_sample", False),
            "streamer": streamer,
        }
        
        # Add sampling parameters if sampling is enabled
        if generation_kwargs["do_sample"]:
            generation_kwargs["temperature"] = exec_config.get("temperature", 1.0)
            generation_kwargs["top_p"] = exec_config.get("top_p", 0.95)
        
        # Start generation in a separate thread with torch.no_grad()               
        def generate_with_no_grad():                                               
          with torch.no_grad():                                                  
              self.model.generate(**generation_kwargs)                           
        
        thread = Thread(target=generate_with_no_grad)                              
        thread.start() 
        
        # Collect generated text
        generated_text = ""
        for text_chunk in streamer:
            generated_text += text_chunk
            yield text_chunk
        
        # Wait for generation to complete
        thread.join()

        # Clean up tensors immediately
        del inputs
        
        # Clear GPU cache if using CUDA
        if self.device == "cuda" and torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        # Return final result
        return TranscriptionResult(
            text=generated_text.strip(),
            confidence=None,
            segments=None,
            metadata={
                "model": exec_config["model_id"],
                "language": exec_config.get("language", "en"),
                "device": self.device,
                "dtype": str(self.dtype),
                "streaming": True,
            }
        )
        
    finally:
        # Clean up temporary file if created
        if temp_file_created:
            try:
                Path(audio_path).unlink()
            except Exception:
                pass

## Testing the Plugin

In [ ]:
# Test basic functionality
plugin = VoxtralHFPlugin()

# Check availability
print(f"Voxtral available: {plugin.is_available()}")
print(f"Plugin name: {plugin.name}")
print(f"Plugin version: {plugin.version}")
print(f"Supported formats: {plugin.supported_formats}")
print(f"Supports streaming: {plugin.supports_streaming()}")

Voxtral available: True
Plugin name: voxtral_hf
Plugin version: 1.0.0
Supported formats: ['wav', 'mp3', 'flac', 'm4a', 'ogg', 'webm', 'mp4', 'avi', 'mov']
Supports streaming: True


In [ ]:
# Test configuration schema
schema = plugin.get_config_schema()
print("Available models:")
for model in schema["properties"]["model_id"]["enum"]:
    print(f"  - {model}")

Available models:
  - mistralai/Voxtral-Mini-3B-2507
  - mistralai/Voxtral-Small-24B-2507


In [ ]:
# Test configuration validation
test_configs = [
    ({"model_id": "mistralai/Voxtral-Mini-3B-2507"}, "Valid config"),
    ({"model_id": "invalid_model"}, "Invalid model"),
    ({"temperature": 2.5}, "Temperature out of range"),
]

for config, description in test_configs:
    is_valid, error = plugin.validate_config(config)
    print(f"{description}: Valid={is_valid}")
    if error:
        print(f"  Error: {error[:100]}")

Valid config: Valid=True
Invalid model: Valid=False
  Error: 'invalid_model' is not one of ['mistralai/Voxtral-Mini-3B-2507', 'mistralai/Voxtral-Small-24B-2507']
Temperature out of range: Valid=False
  Error: 'model_id' is a required property

Failed validating 'required' in schema:
    {'$schema': 'http://j


In [ ]:
# Test initialization
plugin.initialize({"model_id": "mistralai/Voxtral-Mini-3B-2507", "device": "cpu"})
print(f"Current config: {plugin.get_current_config()['model_id']}")

Current config: mistralai/Voxtral-Mini-3B-2507


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()